In [1]:
from pope_model_api import *

layer_name: ['self', 'cross', 'self', 'cross', 'self', 'cross', 'self', 'cross']
layer_name: ['self', 'cross']


2023-12-22 10:52:56.828 | INFO     | pope_model_api:<module>:182 - load Matcher successfully


In [2]:
ckpt, model_type = get_model_info("h")
sam = sam_model_registry[model_type](checkpoint=ckpt)
DEVICE = "cuda"
sam.to(device=DEVICE)
MASK_GEN = SamAutomaticMaskGenerator(sam)
logger.info(f"load SAM model from {ckpt}")
crop_tool = CropImage()
dinov2_model = load_dinov2_model()
dinov2_model.to("cuda:0")

2023-12-22 10:53:06.300 | INFO     | __main__:<module>:6 - load SAM model from weights/sam_vit_h_4b8939.pth


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): DropPath()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): DropPath()
    )
  )
  (n

In [3]:
metrics = dict()
metrics.update({'R_errs':[], 't_errs':[], 'inliers':[], "identifiers":[]})

In [4]:
ROOT_DIR = "data/onepose/"

In [5]:
id2name_dict = {
    1: "ape",
    2: "benchvise",
    4: "camera",
    5: "can",
    6: "cat",
    8: "driller",
    9: "duck",
    10: "eggbox",
    11: "glue",
    12: "holepuncher",
    13: "iron",
    14: "lamp",
    15: "phone",
}

In [6]:
res_table = []

In [7]:
import json
with open("data/pairs/Onepose-test.json") as f:
    dir_list = json.load(f)

In [8]:
for label_idx, test_dict in enumerate(dir_list):
    logger.info(f"Onepose: {label_idx}")
    metrics = dict()
    metrics.update({'R_errs':[], 't_errs':[], 'inliers':[], "identifiers":[]})
    sample_data = dir_list[label_idx]["0"][0]
    label = sample_data.split("/")[0]
    name = label.split("-")[1]
    dir_name = os.path.dirname(sample_data)
    FULL_ROOT_DIR = os.path.join(ROOT_DIR, dir_name)
    recall_image, all_image = 0, 0
    for rotation_key, rotation_list in zip(test_dict.keys(), test_dict.values()):
        for pair_idx,pair_name in enumerate(tqdm(rotation_list)):
            all_image = all_image + 1
            base_name = os.path.basename(pair_name)
            idx0_name = base_name.split("-")[0]
            idx1_name = base_name.split("-")[1]
            image0_name = os.path.join(FULL_ROOT_DIR, idx0_name)
            image1_name = os.path.join(FULL_ROOT_DIR.replace("color", "color"), idx1_name)
            intrinsic_path = image0_name.replace("color", "intrin_ba").replace("png", "txt")
            K0 = np.loadtxt(intrinsic_path, delimiter=' ')
            intrinsic_path = image1_name.replace("color", "intrin_ba").replace("png", "txt")
            K1 = np.loadtxt(intrinsic_path, delimiter=' ')
            image0 = cv2.imread(image0_name)
            ref_torch_image = set_torch_image(image0, center_crop=True)
            ref_fea = get_cls_token_torch(dinov2_model, ref_torch_image)
            image1 = cv2.imread(image1_name)
            image_h, image_w, _ = image1.shape
            t1 = time.time()
            masks = MASK_GEN.generate(image1)
            t2 = time.time()
            similarity_score, top_images  = np.array([0, 0, 0], np.float32), [[], [], []]
            t3 = time.time()
            compact_percent = 0.3
            for xxx, mask in enumerate(masks):
                object_mask = np.expand_dims(mask["segmentation"], -1)
                x0, y0, w, h = mask["bbox"]
                x1, y1 = x0 + w, y0 + h
                x0 -= int(w * compact_percent)
                y0 -= int(h * compact_percent)
                x1 += int(w * compact_percent)
                y1 += int(h * compact_percent)
                box = np.array([x0, y0, x1, y1])
                resize_shape = np.array([y1 - y0, x1 - x0])
                K_crop, K_crop_homo = get_K_crop_resize(box, K1, resize_shape)
                image_crop, _ = get_image_crop_resize(image1, box, resize_shape)
                # object_mask, _ = get_image_crop_resize(object_mask, box, resize_shape)
                box_new = np.array([0, 0, x1 - x0, y1 - y0])
                resize_shape = np.array([512, 512])
                K_crop, K_crop_homo = get_K_crop_resize(box_new, K_crop, resize_shape)
                image_crop, _ = get_image_crop_resize(image_crop, box_new, resize_shape)
                crop_tensor = set_torch_image(image_crop, center_crop=True)
                with torch.no_grad():
                    fea = get_cls_token_torch(dinov2_model, crop_tensor)
                score = F.cosine_similarity(ref_fea, fea, dim=1, eps=1e-8)
                if  (score.item() > similarity_score).any():
                    mask["crop_image"] = image_crop
                    mask["K"] = K_crop
                    mask["bbox"] = box
                    min_idx = np.argmin(similarity_score)
                    similarity_score[min_idx] = score.item()
                    top_images[min_idx] = mask.copy()

            img0 = cv2.cvtColor(image0, cv2.COLOR_BGR2GRAY)
            img0 = torch.from_numpy(img0).float()[None] / 255.
            img0 = img0.unsqueeze(0).cuda()

            matching_score = [[0] for _ in range(len(top_images))]
            for top_idx in range(len(top_images)):
                if "crop_image" not in top_images[top_idx]:
                    continue
                img1 = cv2.cvtColor(top_images[top_idx]["crop_image"], cv2.COLOR_BGR2GRAY)
                img1 = torch.from_numpy(img1).float()[None] / 255.
                img1 = img1.unsqueeze(0).cuda()
                batch = {'image0':img0, 'image1':img1}
                with torch.no_grad():
                    matcher(batch)
                    mkpts0 = batch['mkpts0_f'].cpu().numpy()
                    mkpts1 = batch['mkpts1_f'].cpu().numpy()
                    confidences = batch["mconf"].cpu().numpy()
                conf_mask = np.where(confidences > 0.9)
                matching_score[top_idx] = conf_mask[0].shape[0]
                top_images[top_idx]["mkpts0"] = mkpts0
                top_images[top_idx]["mkpts1"] = mkpts1
                top_images[top_idx]["mconf"] = confidences
            #---------------------------------------------------
            # crop_image = cv2.resize(top_images[np.argmax(matching_score)]["crop_image"], (256, 256))
            # que_image = cv2.resize(image0, (256, 256))
            # image = np.hstack((que_image, crop_image))
            # for top_idx in range(len(top_images)):
            #     crop_image = top_images[top_idx]["crop_image"]
            #     score = matching_score[top_idx]
            #     crop_image = cv2.resize(crop_image, (256, 256))
            #     cv2.putText(crop_image,f'{score}', (100, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
            #     image = np.hstack((image, crop_image))
            # cv2.imwrite(f"segment_anything/crop_images/{idx}.jpg", image)
            #---------------------------------------------------
            t4 = time.time()
            # print(f"t4-t3: object detection:{1000*(t4-t3)} ms")
            pose0_name = image0_name.replace("color", "poses_ba").replace("png", "txt")
            pose1_name = image1_name.replace("color", "poses_ba").replace("png", "txt")
            pose0 = np.loadtxt(pose0_name)
            pose1 = np.loadtxt(pose1_name)
            # pose0 = np.vstack((pose0, np.array([[0, 0, 0, 1]])))
            # pose1 = np.vstack((pose1, np.array([[0, 0, 0, 1]])))
            relative_pose = np.matmul(pose1, inv(pose0))
            t = relative_pose[:3, -1].reshape(1, 3)

            if "crop_image" not in top_images[top_idx]:
                continue
            max_match_idx = np.argmax(matching_score)
            pre_bbox = top_images[max_match_idx]["bbox"]
            mkpts0 = top_images[max_match_idx]["mkpts0"]
            mkpts1 = top_images[max_match_idx]["mkpts1"]
            pre_K = top_images[max_match_idx]["K"]

            _3d_bbox = np.loadtxt(f"{os.path.join(ROOT_DIR, label)}/box3d_corners.txt")
            bbox_pts_3d, _ = project_points(_3d_bbox, pose1[:3,:4], K1)
            bbox_pts_3d = bbox_pts_3d.astype(np.int32)
            x0, y0, w, h = cv2.boundingRect(bbox_pts_3d)
            x1, y1 = x0 + w, y0 + h
            gt_bbox = np.array([x0, y0, x1, y1])
            is_recalled = recall_object(pre_bbox , gt_bbox)
            recall_image = recall_image + int(is_recalled>0.5)
            ret = estimate_pose(mkpts0, mkpts1, K0, pre_K, 0.5, 0.99)
            if ret is not None:
                Rot, t, inliers = ret
                t_err, R_err = relative_pose_error(relative_pose, Rot, t, ignore_gt_t_thr=0.0)
                metrics['R_errs'].append(R_err)
                metrics['t_errs'].append(t_err)
            else:
                metrics['R_errs'].append(90)
                metrics['t_errs'].append(90)
            metrics["identifiers"].append( pair_name )


    print(f"Acc: {recall_image}/{all_image}")
    import pprint
    from src.utils.metrics import (
        aggregate_metrics
    )
    from loguru import logger
    val_metrics_4tb = aggregate_metrics(metrics, 5e-4)
    val_metrics_4tb["AP50"] = recall_image/all_image
    logger.info('\n' + pprint.pformat(val_metrics_4tb))

    res_table.append([f"{name }"] + list(val_metrics_4tb.values()))

2023-12-22 10:53:08.111 | INFO     | __main__:<module>:2 - Onepose: 0
100%|██████████| 46/46 [03:03<00:00,  3.98s/it]
2023-12-22 11:23:04.028 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 437 unique items...
2023-12-22 11:23:04.043 | INFO     | __main__:<module>:144 - 
{'AP50': 0.9157175398633257,
 'R:ACC15': 0.8924485125858124,
 'R:ACC30': 0.9473684210526315,
 'R:auc@15': 0.6724567117927808,
 'R:auc@30': 0.7992823940520922,
 'R:medianErr': 3.031618772478778,
 't:ACC15': 0.7208237986270023,
 't:ACC30': 0.8375286041189931,
 't:auc@15': 0.46158968009051476,
 't:auc@30': 0.6261763158769159,
 't:medianErr': 6.864875770558522}
2023-12-22 11:23:04.044 | INFO     | __main__:<module>:2 - Onepose: 1


Acc: 402/439


100%|██████████| 34/34 [02:15<00:00,  3.99s/it]
2023-12-22 11:42:40.679 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 293 unique items...
2023-12-22 11:42:40.684 | INFO     | __main__:<module>:144 - 
{'AP50': 0.9149659863945578,
 'R:ACC15': 0.9523809523809523,
 'R:ACC30': 0.9897959183673469,
 'R:auc@15': 0.818061042178066,
 'R:auc@30': 0.8959128294560712,
 'R:medianErr': 1.5641893073570556,
 't:ACC15': 0.9251700680272109,
 't:ACC30': 0.9489795918367347,
 't:auc@15': 0.6950977592186868,
 't:auc@30': 0.8183236592466585,
 't:medianErr': 3.093133254731688}
2023-12-22 11:42:40.685 | INFO     | __main__:<module>:2 - Onepose: 2


Acc: 269/294


100%|██████████| 30/30 [02:02<00:00,  4.07s/it]
2023-12-22 11:57:54.934 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 227 unique items...
2023-12-22 11:57:54.938 | INFO     | __main__:<module>:144 - 
{'AP50': 0.8986784140969163,
 'R:ACC15': 0.9295154185022027,
 'R:ACC30': 0.9779735682819384,
 'R:auc@15': 0.7721895057872487,
 'R:auc@30': 0.8601383003919311,
 'R:medianErr': 1.619130975753751,
 't:ACC15': 0.8898678414096917,
 't:ACC30': 0.9427312775330396,
 't:auc@15': 0.6724174502213219,
 't:auc@30': 0.7999073206844854,
 't:medianErr': 3.084918218399967}
2023-12-22 11:57:54.939 | INFO     | __main__:<module>:2 - Onepose: 3


...

Acc: 258/305


100%|██████████| 34/34 [02:16<00:00,  4.00s/it]
2023-12-22 14:00:25.527 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 327 unique items...
2023-12-22 14:00:25.534 | INFO     | __main__:<module>:144 - 
{'AP50': 0.5535168195718655,
 'R:ACC15': 0.9541284403669725,
 'R:ACC30': 0.9908256880733946,
 'R:auc@15': 0.7600014343188971,
 'R:auc@30': 0.8680396930093818,
 'R:medianErr': 2.092557509963824,
 't:ACC15': 0.8899082568807339,
 't:ACC30': 0.9388379204892966,
 't:auc@15': 0.6765830760898394,
 't:auc@30': 0.7998774046735577,
 't:medianErr': 3.232856482420301}
2023-12-22 14:00:25.535 | INFO     | __main__:<module>:2 - Onepose: 9


Acc: 181/327


100%|██████████| 32/32 [02:09<00:00,  4.05s/it]
2023-12-22 14:27:30.652 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 404 unique items...
2023-12-22 14:27:30.659 | INFO     | __main__:<module>:144 - 
{'AP50': 0.7475247524752475,
 'R:ACC15': 0.8589108910891089,
 'R:ACC30': 0.9306930693069307,
 'R:auc@15': 0.6165718156915662,
 'R:auc@30': 0.758987874682682,
 'R:medianErr': 3.800889599710521,
 't:ACC15': 0.6633663366336634,
 't:ACC30': 0.8143564356435643,
 't:auc@15': 0.4208170469084095,
 't:auc@30': 0.5891807791464241,
 't:medianErr': 7.851929138773217}


Acc: 302/404


In [9]:
from tabulate import tabulate
headers = ["Category"] + list(val_metrics_4tb.keys())
all_data = np.array(res_table)[:, 1:].astype(np.float32)
res_table.append(["Avg"] + all_data.mean(0).tolist())
print(tabulate(res_table, \
    headers=headers, tablefmt='fancy_grid'))

╒════════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤══════════╕
│ Category       │   R:auc@15 │   R:auc@30 │   R:ACC15 │   R:ACC30 │   R:medianErr │   t:auc@15 │   t:auc@30 │   t:ACC15 │   t:ACC30 │   t:medianErr │     AP50 │
╞════════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪══════════╡
│ aptamil        │   0.672457 │   0.799282 │  0.892449 │  0.947368 │       3.03162 │   0.46159  │   0.626176 │  0.720824 │  0.837529 │       6.86488 │ 0.915718 │
├────────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼──────────┤
│ jzhg           │   0.818061 │   0.895913 │  0.952381 │  0.989796 │       1.56419 │   0.695098 │   0.818324 │  0.92517  │  0.94898  │       3.09313 │ 0.914966 │
├────────────────┼──────────